Solving exercises in ch7 Deep Learning with Pytorch book

The dataset is zipped, but we can use the cifar10 dataset available in public domain.

In [ ]:
from torchvision import datasets

## Q1.  Use torchvision to implement random cropping of the data.

Lets try and implement random crop using transforms. Lets first see what transforms are available.

In [ ]:
from torchvision import transforms

dir(transforms)

lets initialize a normalize function with values given from the book itself.

In [ ]:
normalize = transforms.Normalize((0.4915, 0.4823, 0.4468),
                         (0.2470, 0.2435, 0.2616))

you can compose a transformation pipeline using transforms.Compose.

In [ ]:
train_transform = transforms.Compose(
                    [transforms.RandomCrop(32,32),
                     transforms.ToTensor(),
                     normalize]
)

lets download the data

In [ ]:
data_path = './'

cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=train_transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=train_transform)

In [ ]:
len(cifar10), len(cifar10_val)

we have 50000 images for training and 10000 for validation

In [ ]:
img, label = cifar10[30000]
# img, label

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_cropped_image(index, number):
    columns = number
    rows = 1
    fig = plt.figure(figsize=(32,32))
    for i in range(1, columns*rows +1):
        img, label = cifar10[index]
        fig.add_subplot(rows, columns, i)
        plt.imshow(img.permute(1,2,0))
    plt.show() 

In [ ]:
get_cropped_image(550, 5)

In [ ]:
get_cropped_image(20345,5)

Random resized crop is definitly as random as it professes out to be. Is it possible to create a classfiier with this? lets find out. We will create a simple classfier that checks between a plane and a bird.

the classes in cifar10.

In [ ]:
cifar10.classes, len(cifar10.classes)

Lets create a simple model.

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
# model = nn.Sequential(
#         nn.Linear(3072, 1024),
#         nn.Tanh(),
#         nn.Linear(1024, 512),
#         nn.Tanh(),
#         nn.Linear(512, 128),
#         nn.Tanh(),
#         nn.Linear(128, 10),
#         nn.LogSoftmax(dim=1)
# )

#simpler
model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, 10),
    nn.LogSoftmax(dim=1)
)

defining hyperparameters

In [ ]:
loss_fn = nn.NLLLoss()
n_epochs = 30
learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(),
                      lr = learning_rate)

In [ ]:
import torch

training

In [ ]:
for epoch in range(n_epochs):
    for img, label in cifar10:
        print("Current label: ", label, end="\r")
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: ", epoch, "Loss: ", float(loss))

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in cifar10_val:
        outputs = model(img.view(-1).unsqueeze(0))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted==labels).sum())

print("Accuracy: ", correct/total)

We can also could have used data loaders for creating batches for iterating over in every epoch.

# Q2. Switch Loss function

We will use MSELoss for testing

In [ ]:
loss_fn = nn.MSELoss()
n_epochs = 1
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

we will create a data loader this time.

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=10,shuffle=True )

In [ ]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: ", epoch, " loss: ", float(loss))

Checking on validation set

In [ ]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=10, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted==labels).sum())

print("Accuracy: ", correct/total)